In [1]:
import warnings
warnings.filterwarnings("ignore")

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA,ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
import wget

In [2]:
file_name = 'companyPolicies.txt'
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt"
wget.download(URL, out=file_name)
print("Document Downloaded")


Document Downloaded


In [3]:
loader = TextLoader('companyPolicies.txt')
documents = loader.load_and_split()
print(f"Documents: {len(documents)}")

Documents: 5


In [4]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_49143/3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_49143/3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [5]:
docsearch = Chroma.from_documents(documents, embeddings)
print("documents ingested")

documents ingested


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.3)

In [7]:
prompt="""
you are an helpful assitant and use the information from the document to answer the question at the end. if you do not know the answer, just say Don't know.
do not try to make up the answer.
{context}

Question: {question}

"""
promtp_template = PromptTemplate(template=prompt, input_variables=["context","question"])

chain_type_kwargs = {"prompt": promtp_template}

In [8]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=False
)

In [32]:
while True:
    query = input('Enter your Query')
    if query in ["bye","exit"]:
        break
    print(qa.invoke(query)['result'])

Based on the Mobile Phone Policy, the key words are:

- Mobile
- Phone
- Security
- Confidentiality
- Cost
- Management
- Compliance
- Monitoring
- Consequences
- Termination
- Privileges
- Lost
- Stolen
- Devices
- IT
- Department
- Supervisor
According to the Smoking Policy, No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are owned or leased, to maintain the condition and cleanliness of these vehicles.
Don't know. There is no information in the provided document about eating in company vehicles.


In [30]:
response = qa.invoke("Summarize this doc for me in 20 words only")

## Conversational chain type `Not working now`

In [9]:
#memory = ConversationalRetrievalChain(memory_key="chat_history", return_message=True)
memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

/tmp/ipykernel_49143/2068878438.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)


In [15]:
memoryQA = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory=memory,
    get_chat_history=lambda h: h,
    return_source_documents=False
)

history = []

while True:
    query = input("Enter your Query: ")
    if query.lower() in ["bye", "exit"]:
        break

    response = memoryQA.invoke({
        "question": query,
        "chat_history": history
    })

    print(f"Answer: {response['answer']}")
    history.append((query, response['answer']))


Answer: I'm an AI assistant, and I'm here to provide information and answer questions based on the context provided to me. In this case, I'm a representative of a company that has a set of policies outlined in the text you provided, including Health and Safety Policy, Anti-discrimination and Harassment Policy, Discipline and Termination Policy, and others.

I don't have a personal identity, but I'm here to help answer your questions and provide information about the company's policies and procedures. If you have any specific questions or concerns, feel free to ask!
Answer: Based on the provided information, here's a summary of the company's policies and procedures:

**Code of Conduct:**

* Emphasizes integrity, respect, accountability, safety, and environmental responsibility
* Outlines expectations for employee behavior and conduct
* Encourages a culture of transparency, inclusivity, and social responsibility

**Recruitment Policy:**

* Emphasizes equal opportunity employment and dive